In [1]:
import numpy as np

In [2]:
data = np.load("C:\\Users\\Семья\\Documents\\vibro_analitics\\datasets\\acceleration.npz")

In [3]:
X = data["X"]
y = data["y"]

In [4]:
len(y)

2808

In [5]:
X.shape

(2808, 24000)

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_full, X_test, y_full, y_test = train_test_split(X, y, test_size=0.2, random_state=239)

In [8]:
X_train, X_val, y_train, y_val = train_test_split(X_full, y_full, test_size=0.2, random_state=239)

In [9]:
from sklearn.decomposition import PCA

In [10]:
pca = PCA(n_components=50)

In [11]:
pca.fit(X_train)

PCA(n_components=50)

In [12]:
sum(pca.explained_variance_ratio_)

0.3540928125148639

In [13]:
X_train_pca = pca.transform(X_train)
X_val_pca = pca.transform(X_val)
X_test_pca = pca.transform(X_test)

In [14]:
from sklearn.linear_model import LinearRegression

In [15]:
model = LinearRegression()

In [16]:
model.fit(X_train_pca, y_train)

LinearRegression()

In [17]:
y_train_pred = model.predict(X_train_pca)
y_val_pred = model.predict(X_val_pca)
y_test_pred = model.predict(X_test_pca)

In [18]:
from sklearn.metrics import r2_score

In [19]:
r2_score(y_train, y_train_pred)

0.009251456187502782

In [20]:
r2_score(y_val, y_val_pred)

-0.021244784534292904

In [21]:
r2_score(y_test, y_test_pred)

-0.02630089937199509

In [22]:
pca = PCA(n_components=250)

In [23]:
pca.fit(X_train)
X_train_pca = pca.transform(X_train)
X_val_pca = pca.transform(X_val)
X_test_pca = pca.transform(X_test)

In [24]:
import catboost

In [25]:
model = catboost.CatBoostRegressor(random_state=239, use_best_model=True)

In [26]:
model.fit(X_train_pca, y_train, eval_set=catboost.Pool(data=X_val_pca, label=y_val))

Learning rate set to 0.055824
0:	learn: 1.7101284	test: 1.6712358	best: 1.6712358 (0)	total: 176ms	remaining: 2m 55s
1:	learn: 1.7058732	test: 1.6702141	best: 1.6702141 (1)	total: 199ms	remaining: 1m 39s
2:	learn: 1.7017562	test: 1.6701503	best: 1.6701503 (2)	total: 227ms	remaining: 1m 15s
3:	learn: 1.6985409	test: 1.6702338	best: 1.6701503 (2)	total: 252ms	remaining: 1m 2s
4:	learn: 1.6956818	test: 1.6699057	best: 1.6699057 (4)	total: 276ms	remaining: 54.9s
5:	learn: 1.6925785	test: 1.6685160	best: 1.6685160 (5)	total: 299ms	remaining: 49.5s
6:	learn: 1.6895191	test: 1.6676584	best: 1.6676584 (6)	total: 323ms	remaining: 45.8s
7:	learn: 1.6857162	test: 1.6674134	best: 1.6674134 (7)	total: 347ms	remaining: 43s
8:	learn: 1.6832912	test: 1.6666179	best: 1.6666179 (8)	total: 370ms	remaining: 40.7s
9:	learn: 1.6776019	test: 1.6671662	best: 1.6666179 (8)	total: 394ms	remaining: 39s
10:	learn: 1.6751788	test: 1.6666770	best: 1.6666179 (8)	total: 418ms	remaining: 37.6s
11:	learn: 1.6709681	tes

In [27]:
y_train_pred = model.predict(X_train_pca)
y_val_pred = model.predict(X_val_pca)
y_test_pred = model.predict(X_test_pca)

In [28]:
r2_score(y_train, y_train_pred)

0.7492782259128774

In [29]:
r2_score(y_val, y_val_pred)

0.028303211892475644

In [30]:
r2_score(y_test, y_test_pred)

0.04498869870232647

In [33]:
from scipy import fft
from scipy.ndimage import uniform_filter1d

def preprocess(X):
    X_fft = np.abs(fft.rfft(X[:, :2400]))[:,1:]
    X_fft /= np.max(X_fft, axis=1, keepdims=1)
    Y = uniform_filter1d(X_fft, size=10)
    Y = Y/np.max(Y, axis=1, keepdims=True)
    return Y

In [36]:
X_train_hat = preprocess(X_train)
X_val_hat = preprocess(X_val)
X_test_hat = preprocess(X_test)

In [37]:
X_train_hat.shape

(1796, 1200)

In [45]:
model = catboost.CatBoostRegressor(random_state=239, use_best_model=True)

In [47]:
model.fit(X_train_hat, y_train, eval_set=catboost.Pool(data=X_val_hat, label=y_val))

Learning rate set to 0.055824
0:	learn: 1.6387357	test: 1.6022328	best: 1.6022328 (0)	total: 127ms	remaining: 2m 6s
1:	learn: 1.5685874	test: 1.5319968	best: 1.5319968 (1)	total: 242ms	remaining: 2m
2:	learn: 1.5014396	test: 1.4702804	best: 1.4702804 (2)	total: 348ms	remaining: 1m 55s
3:	learn: 1.4450200	test: 1.4135701	best: 1.4135701 (3)	total: 454ms	remaining: 1m 53s
4:	learn: 1.3884787	test: 1.3571126	best: 1.3571126 (4)	total: 569ms	remaining: 1m 53s
5:	learn: 1.3330956	test: 1.3035449	best: 1.3035449 (5)	total: 681ms	remaining: 1m 52s
6:	learn: 1.2796732	test: 1.2551646	best: 1.2551646 (6)	total: 797ms	remaining: 1m 53s
7:	learn: 1.2273312	test: 1.2064325	best: 1.2064325 (7)	total: 917ms	remaining: 1m 53s
8:	learn: 1.1817072	test: 1.1629985	best: 1.1629985 (8)	total: 1.03s	remaining: 1m 53s
9:	learn: 1.1392562	test: 1.1244693	best: 1.1244693 (9)	total: 1.15s	remaining: 1m 53s
10:	learn: 1.0995807	test: 1.0845076	best: 1.0845076 (10)	total: 1.26s	remaining: 1m 53s
11:	learn: 1.057

In [48]:
y_train_hat = model.predict(X_train_hat)
y_val_hat = model.predict(X_val_hat)
y_test_hat = model.predict(X_test_hat)

In [49]:
r2_score(y_train, y_train_hat)

0.9998509082079324

In [50]:
r2_score(y_val, y_val_hat)

0.9717139503118839

In [51]:
r2_score(y_test, y_test_hat)

0.9798181669923597